## Robust Query Client
This notebook is all about using the SpotifyAPI class to obtain an access token and making more complex search query requests to Spotify.

In [1]:
import requests
import base64
import datetime
import json
from urllib.parse import urlencode

In [2]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.perform_auth()
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded String.
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_secret == None or client_id == None:
            raise Exception("client_id and client_secret required.")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return { 
        "Authorization" : f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
        "grant_type" : "client_credentials"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_headers(self):
        access_token = self.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}
        return headers
    
    def get_resource(self, lookup_id, resource_type="track", version="v1"):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_artist(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="artists")
    
    def get_album(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="albums")
    
    def get_track_features(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="audio-features")
    
    def base_search(self, query_params):
        lookup_url = f"https://api.spotify.com/v1/search?{query_params}"
        headers = self.get_resource_headers()
        
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if not r.status_code in range(200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, search_type="artist", market_type="GB"):
        new_query=""
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            for key,value in query.items():
                new_query += f"{key}:{value} "
        query_params = urlencode({"q" : new_query, "type" : search_type.lower(), "market" : market_type})
        return self.base_search(query_params)
    
    def key_convert(self, key: int, mode: int=None) -> str:
        if key < 0 or key > 10:
            raise Exception("Invalid key value entered - bad Python joke")
        keys = {0: "C", 1: "Db", 2: "D", 3: "Eb",
                4: "Eb", 5: "F", 6: "Gb", 7: "G",
                8: "Ab", 9: "B", 10: "Bb"}
        if mode == None:
            return keys[key]
        elif mode == 1:
            return f"{keys[key]} Major"
        else:
            return f"{keys[key]} Minor"

    def time_signature_convert(self, time_signature: int) -> str:
        if time_signature < 0 or time_signature > 10:
            return "invalid key"
        keys = {0: "C", 1: "Db", 2: "D", 3: "Eb",
                4: "Eb", 5: "F", 6: "Gb", 7: "G",
                8: "Ab", 9: "B", 10: "Bb"}
        return keys[key]
        

### SEARCH
Let's try searching for something!

In [3]:
client_id = "a3aadf7f4d184af0af9d849d96e89b12"
client_secret = ""

In [4]:
spotify = SpotifyAPI(client_id, client_secret)
json_data = spotify.search({"track": "money", "artist": "pink floyd"}, search_type="track")
tracks = json_data['tracks']['items']



https://api.spotify.com/v1/search?q=track%3Amoney+artist%3Apink+floyd+&type=track&market=GB
200


In [14]:
for i in range(len(tracks)):
    print(f"Track id: {tracks[i]['id']}")
    print(f"Track name: {tracks[i]['name']}")
    print(f"Artist: {tracks[i]['artists'][0]['name']}")
    print(f"Track URL: {tracks[i]['external_urls']['spotify']}")
    print(f"Image URL: {tracks[i]['album']['images'][0]['url']}")
    print("\n")

Track id: 7Gx2q0ueNwvDp2BOZYGCMO
Track name: Money - 2011 Remastered Version
Artist: Pink Floyd
Track URL: https://open.spotify.com/track/7Gx2q0ueNwvDp2BOZYGCMO
Image URL: https://i.scdn.co/image/ab67616d0000b27331c57b302f0e3aca46ab7561


Track id: 6FjDEJNmx4XWYilqhTs7G9
Track name: Money - Live
Artist: Pink Floyd
Track URL: https://open.spotify.com/track/6FjDEJNmx4XWYilqhTs7G9
Image URL: https://i.scdn.co/image/ab67616d0000b27369f4b7cda08f4ed73cc20474


Track id: 77IH1EBjOSSBo30ie5XwEo
Track name: Money - Live
Artist: Pink Floyd
Track URL: https://open.spotify.com/track/77IH1EBjOSSBo30ie5XwEo
Image URL: https://i.scdn.co/image/ab67616d0000b2737946212e06e309d833b7999e


Track id: 1WakuskS7SyaWnXqU4cQS7
Track name: Money - 2001 Remastered Version
Artist: Pink Floyd
Track URL: https://open.spotify.com/track/1WakuskS7SyaWnXqU4cQS7
Image URL: https://i.scdn.co/image/ab67616d0000b273c1005b1d9b7ba2c343f2928f


Track id: 4uUSO5Qd2ZqfVmVLMHvnl9
Track name: Money - 2019 Remix, Live
Artist: Pink

In [11]:
# Make Track Analysis Request for BPM etc.
track_data = spotify.get_track_features("7Gx2q0ueNwvDp2BOZYGCMO")

key = spotify.key_convert(track_data['key'], track_data['mode'])
tempo = int(track_data['tempo'])

print(f"Key: {key}")
print(f"Tempo: {tempo} bpm")


Key: B Major
Tempo: 124 bpm
